## SI649 Project 2
### liujr
### Jiaren Liu
#### vis1

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
alt.themes.enable('fivethirtyeight')

ThemeRegistry.enable('fivethirtyeight')

In [13]:
math = pd.read_csv('ny-math-results-2013-2019-public-all.csv', index_col=0)
math = math[(math['School Name'] == 'P.S. 307 DANIEL HALE WILLIAMS') | 
            (math['School Name'] == 'P.S. 008 ROBERT FULTON')]
math = math[math['Grade'] == 'All Grades']
math = math[['School Name', 'Year', 'Mean Scale Score', '% Level 1', '% Level 2', '% Level 3+4']]
math = math.rename(columns={
    'Mean Scale Score': 'Mean Scale Score',
    '% Level 1': 'L1',
    '% Level 2': 'L2',
    '% Level 3+4': 'L3,4'
})
math['Year'] = pd.to_datetime(math['Year'], format='%Y')
math['Mean Scale Score'] = pd.to_numeric(math['Mean Scale Score'], errors='coerce')
math['L1'] = pd.to_numeric(math['L1'], errors='coerce')
math['L2'] = pd.to_numeric(math['L2'], errors='coerce')
math['L3,4'] = pd.to_numeric(math['L3,4'], errors='coerce')
math = math[math['Year'].dt.year < 2018]
math['School Name'] = ['P.S 307']*5 + ['P.S 8']*5
math['subject'] = 'math'
ela = pd.read_csv('ny-ela-results-2013-2019-public-all.csv', index_col=0)
ela = ela[(ela['School Name'] == 'P.S. 307 DANIEL HALE WILLIAMS') | 
          (ela['School Name'] == 'P.S. 008 ROBERT FULTON')]
ela = ela[ela['Grade'] == 'All Grades']
ela = ela[['School Name', 'Year', 'Mean Scale Score', '% Level 1', '% Level 2', '% Level 3+4']]
ela = ela.rename(columns={
    'Mean Scale Score': 'Mean Scale Score',
    '% Level 1': 'L1',
    '% Level 2': 'L2',
    '% Level 3+4': 'L3,4'
})
ela['Year'] = pd.to_datetime(ela['Year'], format='%Y')
ela['Mean Scale Score'] = pd.to_numeric(ela['Mean Scale Score'], errors='coerce')
ela['L1'] = pd.to_numeric(ela['L1'], errors='coerce')
ela['L2'] = pd.to_numeric(ela['L2'], errors='coerce')
ela['L3,4'] = pd.to_numeric(ela['L3,4'], errors='coerce')
ela = ela[ela['Year'].dt.year < 2018]
ela['School Name'] = ['P.S 307']*5 + ['P.S 8']*5
ela['subject'] = 'ela'
data = pd.concat([math, ela])
data['Year'] = data['Year'].dt.year

/var/folders/gp/bnf8n57s1nlgccs8xl1kk3mc0000gn/T/ipykernel_9555/1981373183.py:20: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ela = pd.read_csv('ny-ela-results-2013-2019-public-all.csv', index_col=0)


In [14]:
subject_param = alt.param(
    value="math",
    name="subject_param",
    bind=alt.binding_select(options=["math", "ela"], name="Select Subject: ")
) 
school_param = alt.param(
    value="P.S 307",
    name="school_param",
    bind=alt.binding_select(options=["P.S 307", "P.S 8"], name="Select School: ")
)
base = alt.Chart(data).transform_filter(
    "datum.subject == subject_param && datum['School Name'] == school_param"
)
folded = base.transform_fold(
    ['L1', 'L2', 'L3,4'],
    as_=['Level', 'Percentage']
)
chart = folded.mark_bar(orient='horizontal').encode(
    y=alt.Y("Year:N", title="Year"),
    x=alt.X("Percentage:Q", stack="normalize", axis=alt.Axis(format='%')),
    color=alt.Color("Level:N", title="Level"),
    tooltip=[
        alt.Tooltip("School Name:N", title="School"),
        alt.Tooltip("Year:N", title="Year"),
        alt.Tooltip("Level:N", title="Level"),
        alt.Tooltip("Percentage:Q", format=".2f", title="Percentage of Students in this Level")
    ]
).add_params(
    subject_param, school_param
).properties(
    width=400,
    height=300,
    title=alt.TitleParams(
        text="Percentage of Students in Each Level by Year",
        subtitle="Select a subject and a school from two schools and two subjects",
        anchor="start",
        fontSize=20
    )
)
chart.save('int_vis_1.html')
chart

alt.Chart(...)

This interactive stacked bar chart displays the distribution of test score levels (Level 1, Level 2, and Level 3+4) for math and ELA over multiple years for selected schools. Users can choose the subject (math or ELA) and the school (P.S. 307 or P.S. 8) via dropdown menus, which dynamically filter the data displayed in the chart. Hovering over each bar reveals tooltips with detailed information such as the school name, year, specific level, and the corresponding percentage.

### vis2

In [15]:
private = pd.read_csv('NCES_private_clean.csv', index_col=0)
private_1 = private[['State Name [Private School] Latest available year','Percentage of Black Students [Private School] 2015-16','Percentage of White Students [Private School] 2015-16']]
private_1 = private_1.dropna(subset=["State Name [Private School] Latest available year", 
                       "Percentage of Black Students [Private School] 2015-16", 
                       "Percentage of White Students [Private School] 2015-16"])
private_1["Percentage of Black Students [Private School] 2015-16"] = pd.to_numeric(private_1["Percentage of Black Students [Private School] 2015-16"], errors="coerce")
private_1["Percentage of White Students [Private School] 2015-16"] = pd.to_numeric(private_1["Percentage of White Students [Private School] 2015-16"], errors="coerce")
private_1["Highly Segregated"] = (private_1["Percentage of Black Students [Private School] 2015-16"] >= 80) | (private_1["Percentage of White Students [Private School] 2015-16"] >= 80)
df_state = private_1.groupby("State Name [Private School] Latest available year").agg(
    total_schools=("Highly Segregated", "count"),
    segregated_schools=("Highly Segregated", "sum")
).reset_index()
df_state["segregated_percentage"] = (df_state["segregated_schools"] / df_state["total_schools"]) * 100
from vega_datasets import data

data_df = df_state.rename(columns={
    "State Name [Private School] Latest available year": "state",
    "segregated_percentage": "percentage"
})

state_id_map = {
    "ALABAMA": "01",
    "ALASKA": "02",
    "ARIZONA": "04",
    "ARKANSAS": "05",
    "CALIFORNIA": "06",
    "COLORADO": "08",
    "CONNECTICUT": "09",
    "DELAWARE": "10",
    "DISTRICT OF COLUMBIA": "11",
    "FLORIDA": "12",
    "GEORGIA": "13",
    "HAWAII": "15",
    "IDAHO": "16",
    "ILLINOIS": "17",
    "INDIANA": "18",
    "IOWA": "19",
    "KANSAS": "20",
    "KENTUCKY": "21",
    "LOUISIANA": "22",
    "MAINE": "23",
    "MARYLAND": "24",
    "MASSACHUSETTS": "25",
    "MICHIGAN": "26",
    "MINNESOTA": "27",
    "MISSISSIPPI": "28",
    "MISSOURI": "29",
    "MONTANA": "30",
    "NEBRASKA": "31",
    "NEVADA": "32",
    "NEW HAMPSHIRE": "33",
    "NEW JERSEY": "34",
    "NEW MEXICO": "35",
    "NEW YORK": "36",
    "NORTH CAROLINA": "37",
    "NORTH DAKOTA": "38",
    "OHIO": "39",
    "OKLAHOMA": "40",
    "OREGON": "41",
    "PENNSYLVANIA": "42",
    "RHODE ISLAND": "44",
    "SOUTH CAROLINA": "45",
    "SOUTH DAKOTA": "46",
    "TENNESSEE": "47",
    "TEXAS": "48",
    "UTAH": "49",
    "VERMONT": "50",
    "VIRGINIA": "51",
    "WASHINGTON": "53",
    "WEST VIRGINIA": "54",
    "WISCONSIN": "55",
    "WYOMING": "56"
}
data_df["id"] = data_df["state"].map(state_id_map)
data_df["id"] = data_df["id"].astype(int)
us_states = alt.topo_feature(data.us_10m.url, feature='states')


In [16]:
state_select = alt.selection_multi(
    fields=["id"],
    toggle=True,
    empty="all"
)
us_states = alt.topo_feature(data.us_10m.url, feature='states')
data_map = (
    alt.Chart(us_states)
    .mark_geoshape(
        stroke='white',
        strokeWidth=0.5
    )
    .transform_lookup(
        lookup='id',
        from_=alt.LookupData(
            data_df,
            key='id',
            fields=["state", "percentage"]
        )
    )
    .encode(
        color=alt.condition(
            state_select,
            alt.Color("percentage:Q", title="Percentage", scale=alt.Scale(scheme="blues")),
            alt.value("lightgray")
        ),
        tooltip=[
            alt.Tooltip("state:N", title="State"),
            alt.Tooltip("percentage:Q", title="Segregated %", format=".2f")
        ]
    )
    .project(type='albersUsa')
    .properties(
        width=700, 
        height=450,
        title=alt.TitleParams(
            text="Percentage of Highly Segregated Private Schools by State",
            subtitle="Percentage of schools with 80% or more black or white students in selected area",
            anchor="start",
            fontSize=20
        )
    )
    .add_params(state_select)
)
annot_df = pd.DataFrame({
    'state': ['NEW YORK'],
    'latitude': [43],
    'longitude': [-76.0],
    'label': ['NEW YORK']
})
label_layer = (
    alt.Chart(annot_df)
    .mark_text()
    .encode(
        longitude='longitude:Q',
        latitude='latitude:Q',
        text='label:N'
    )
    .project(type='albersUsa')
)
avg_text = alt.Chart(data_df).transform_filter(state_select) \
    .transform_aggregate(avg_percentage='mean(percentage)') \
    .transform_calculate(
        avg_label="'selected area average: ' + format(datum.avg_percentage, '.2f')"
    ) \
    .mark_text(
        align='center',
        baseline='middle',
        fontSize=20,
        fontWeight='bold'
    ).encode(
        x=alt.value(350),
        y=alt.value(30),
        text=alt.Text('avg_label:N'),
        color=alt.Color("avg_percentage:Q", scale=alt.Scale(scheme='redblue'))
    ).properties(
        width=700,
        height=450
    )

final_chart_1 = data_map + label_layer + avg_text

final_chart_1.save('int_vis_2.html')
final_chart_1

/Users/wuhuan/Documents/WuHuan/UMICH/2025WN/SI649/.venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/wuhuan/Documents/WuHuan/UMICH/2025WN/SI649/.venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:405: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


alt.LayerChart(...)

This interactive U.S. map shows the percentage of segregated private schools by state, with color intensity reflecting the level of segregation. Users can select one or more states by clicking on them—selected states are highlighted in a color scheme (with unselected states appearing in light gray). The map calculates and displays, in real time, the average segregation percentage for the selected states as a text label on the map. To clear the selection, users simply click on an empty area of the map. Hovering over a state provides a tooltip with the state’s name and its segregation percentage.